In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import cvxpy as cp
import polytope as pc
import matplotlib.pyplot as plt

from evanqp import QPProblem, Polytope, Verifier
from utils import dlqr

In [3]:
class DoubleIntegrator(QPProblem):
    def __init__(self, N=10):
        self.N = N

        n = 2
        m = 1

        # Double Integrator
        self.A = np.array([[1, 1], [0, 1]])
        self.B = np.array([[0], [1]])
        
        # Weights
        self.Q = np.diag([1, 1])
        self.R = np.array([[0.1]])
        self.K, self.P, _ = dlqr(self.A, self.B, self.Q, self.R)
        
        # Constraints
        self.x_max = np.array([10.0, 10.0])
        self.x_min = np.array([-10.0, -10.0])
        self.u_max = 1.0
        self.u_min = -1.0
        
        # Terminal Set computation
        # state constraints
        Hx = np.vstack((np.eye(n), -np.eye(n)))
        hx = np.concatenate((self.x_max, -self.x_min))
        # input constraints
        Hu = np.vstack((np.eye(m), -np.eye(m)))
        hu = np.array([self.u_max, -self.u_min])
        # closed loop dynamics
        Ak = self.A - self.B @ self.K
        # state & input constraints
        HH = np.vstack((Hx, -Hu @ self.K))
        hh = np.concatenate((hx, hu))
        # compute maximal invariant set
        O = pc.Polytope(HH, hh)
        while True:
            O_prev = O
            # pre-set
            O = O.intersect(pc.Polytope(O.A @ Ak, O.b))
            if O == O_prev:
                break
        self.F, self.f = O.A, O.b

        self.x0 = cp.Parameter(n, name='x0')
        self.xN = cp.Parameter(n, name='xN')
        self.x = cp.Variable((N + 1, n), name='x')
        self.u = cp.Variable((N, m), name='u')
        
        objective = cp.quad_form(self.x[N, :], self.P)
        constraints = [self.x0 == self.x[0, :], self.xN == self.x[N, :]]
        for i in range(N):
            objective += cp.quad_form(self.x[i, :], self.Q) + cp.quad_form(self.u[i, :], self.R)
            constraints += [self.x[i + 1, :] == self.A @ self.x[i, :] + self.B @ self.u[i, :]]
            constraints += [self.x_min <= self.x[i, :], self.x[i, :] <= self.x_max]
            constraints += [self.u_min <= self.u[i, :], self.u[i, :] <= self.u_max]

        self.objective = cp.Minimize(objective)
        self._problem = cp.Problem(self.objective, constraints)

    def problem(self):
        return self._problem

    def parameters(self):
        return [self.x0]

    def variables(self):
        return [self.xN]

    def solve(self, x0):
        self.x0.value = x0
        self._problem.solve(solver=cp.GUROBI)

        solution = {self.u0: self.u0.value,
                    self.u: self.u.value,
                    self.x: self.x.value,
                    self.objective: self.objective.value}
        return solution

In [4]:
mpc_controller = DoubleIntegrator()

In [5]:
parameter_set = Polytope(np.array([[1, 0], [-1, 0], [0, 1], [0, -1]]), np.array([10, 10, 10, 10]))

In [6]:
verifier = Verifier(parameter_set, mpc_controller)

In [8]:
poly = Polytope(mpc_controller.F, mpc_controller.f)
verifier.variables_in_polytope(poly)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Parameter Threads unchanged
   Value: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 192 rows, 250 columns and 432 nonzeros
Model fingerprint: 0xbc94d802
Model has 120 general constraints
Variable types: 190 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+00]
  Objective range  [4e-01, 9e-01]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+01]
Presolve removed 45 rows and 25 columns
Presolve time: 0.00s
Presolved: 147 rows, 225 columns, 392 nonzeros
Presolved model has 76 SOS constraint(s)
Variable types: 149 continuous, 76 integer (76 binary)

Root relaxation: objective 2.649255e+00, 71 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I

False

In [9]:
poly = Polytope(mpc_controller.F, mpc_controller.f)
Verifier.min_optimal_mpc_horizon(parameter_set, lambda N: DoubleIntegrator(N=N), poly)

Checking N = 1
Checking N = 2
Checking N = 4
Checking N = 8
Checking N = 16
Checking N = 12
Checking N = 10
Checking N = 11


11